In [1]:
import wandb
import numpy as np
import torch
import lightning
import copy
from pathlib import Path
from model_fusion.config import BASE_DATA_DIR, CHECKPOINT_DIR
from model_fusion.datasets import DataModuleType
from model_fusion.models import ModelType
from model_fusion.models.lightning import BaseModel 
from Experiments import lmc_experiment
from model_fusion import lmc_utils
from Experiments import baselines_experiment
from Experiments import otfusion_experiment
from Experiments import pyhessian_experiment
from model_fusion.train import setup_training, setup_testing


# set seed for numpy based calculations
NUMPY_SEED = 100
np.random.seed(NUMPY_SEED)

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\__init__.py:11: FutureWarning: In the future `np.bool` will be defined as the corresponding NumPy scalar.
  if not hasattr(numpy, tp_name):
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pl_bolts\models\self_supervised\amdim\amdim_module.py:34: UnderReviewWarning: The feature generate_power_seq is currently marked under review. The compatibility with other Lightning projects is not guaranteed and API may change at any time. The API and functionality may change without warning in future releases. More details: https://lightning-bolts.readthedocs.io/en/latest/stability.html
  "lr_options": generate_power_seq(LEARNING_RATE_CIFAR, 11),
c:\Users\filos\OneDrive\Desk

In [2]:
print("------- Loading models -------")

# select wandb run names
runA = '33kyx0p1'
runB = '3ezo4au5'#same init

api = wandb.Api()
run = api.run(f'model-fusion/Model Fusion/{runA}')

print(run.config)

batch_size = run.config['datamodule_hparams'].get('batch_size')

datamodule_type_str = run.config['datamodule_type'].split('.')[1].lower()
datamodule_type = DataModuleType(datamodule_type_str)
datamodule_hparams = run.config['datamodule_hparams']
datamodule_hparams['data_augmentation'] = False

model_type_str = run.config['model_type'].split('.')[1].lower()
model_type = ModelType(model_type_str)

model_hparams = run.config['model_hparams']

print(datamodule_hparams)
print(model_hparams)

checkpointA = f'model-fusion/Model Fusion/model-{runA}:best_k'
checkpointB = f'model-fusion/Model Fusion/model-{runB}:best_k'

run = wandb.init()

artifact = run.use_artifact(checkpointA, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelA = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")

artifact = run.use_artifact(checkpointB, type='model')
artifact_dir = artifact.download(root=CHECKPOINT_DIR)
modelB = BaseModel.load_from_checkpoint(Path(artifact_dir)/"model.ckpt")


------- Loading models -------


Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


{'lr': 0.1, 'model': None, 'momentum': 0.9, 'data_seed': 42, 'optimizer': 'sgd', 'max_epochs': 200, 'min_epochs': 50, 'model_seed': 42, 'model_type': 'ModelType.VGG11', 'loss_module': 'CrossEntropyLoss', 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'model_hparams': {'bias': False, 'num_classes': 10, 'num_channels': 3}, 'early_stopping': True, 'datamodule_type': 'DataModuleType.CIFAR10', 'lr_decay_factor': 0.1, 'lightning_params': {'lr': 0.1, 'momentum': 0.9, 'data_seed': 42, 'optimizer': 'sgd', 'model_seed': 42, 'lr_scheduler': 'plateau', 'weight_decay': 0.0001, 'lr_decay_factor': 0.1, 'lr_monitor_metric': 'val_loss'}, 'lr_monitor_metric': 'val_loss', 'datamodule_hparams': {'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': True}, 'model_hparams/bias': False, 'model_hparams/num_classes': 10, 'model_hparams/num_channels': 3}
{'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': False}
{'bias': False, 'num_classes': 10, 'num_channels': 3}


wandb: Currently logged in as: frambelli (model-fusion). Use `wandb login --relogin` to force relogin


wandb: Downloading large artifact model-33kyx0p1:best_k, 982.65MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:4.8
wandb: Downloading large artifact model-3ezo4au5:best_k, 982.65MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:3.4


In [ ]:
# LMC barrier
print("------- Computing LMC barrier before alignment-------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA,
    modelB=modelB,
    granularity=21
)

In [3]:
# Baselines (prediction ensembling, vanilla averaging)
print("------- Computing baselines -------")

wandb_tag = f'baselines-{runA}-{runB}'

vanilla_averaging_model = baselines_experiment.run_baselines(
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
)

------- Computing baselines -------
------- Prediction based ensembling -------
------- Naive ensembling of weights -------
------- Evaluating baselines -------


c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\pytorch_lightning\loggers\wandb.py:395: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


You are using a CUDA device ('NVIDIA GeForce RTX 4060 Laptop GPU') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


------- Evaluating base models -------
Testing DataLoader 0: 100%|██████████| 10/10 [00:03<00:00,  2.55it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8693000078201294
        val_loss            0.5114084482192993
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing DataLoader 0: 100%|██████████| 10/10 [00:02<00:00,  4.11it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.8587999939918518
        val_loss            0.5021435618400574
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
------- Evaluating prediction ensembling -------


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Test set: Average loss: 0.4270, Accuracy: 88.25%
------- Evaluating vanilla averaging -------
Testing DataLoader 0: 100%|██████████| 10/10 [00:01<00:00,  5.03it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.21570000052452087
        val_loss            2.3024399280548096
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁▁▁
trainer/global_step,▁▁▁
val_accuracy,██▁
val_loss,▁▁█
epoch,0
trainer/global_step,0
val_accuracy,0.2157
val_loss,2.30244


In [4]:
# OT model fusion + eval aligned model 
print("------- Computing model fusion -------")

wandb_tag = f"ot_model_fusion-{runA}-{runB}"

#modelA.to('cpu')
#modelB.to('cpu')

ot_fused_model, modelA_aligned = otfusion_experiment.run_otfusion(
    batch_size=batch_size,
    datamodule_type=datamodule_type,
    datamodule_hparams=datamodule_hparams,
    model_type=model_type, 
    model_hparams=model_hparams,
    modelA=modelA,
    modelB=modelB,
    wandb_tag=wandb_tag,
    is_vgg=True
)


Seed set to 42


------- Computing model fusion -------
------- Setting up parameters -------
{'seed': 42, 'data_dir': 'data', 'batch_size': 512, 'data_augmentation': False}
The parameters are: 
 {'eval_aligned': True, 'num_models': 2, 'width_ratio': 1, 'handle_skips': False, 'exact': True, 'activation_seed': 21, 'activation_histograms': True, 'ground_metric': 'euclidean', 'ground_metric_normalize': 'none', 'same_model': False, 'geom_ensemble_type': 'acts', 'act_num_samples': 200, 'skip_last_layer': False, 'skip_last_layer_type': 'average', 'softmax_temperature': 1, 'past_correction': True, 'correction': True, 'normalize_acts': False, 'normalize_wts': False, 'activation_normalize': False, 'center_acts': False, 'prelu_acts': False, 'pool_acts': False, 'pool_relu': False, 'importance': None, 'proper_marginals': False, 'not_squared': True, 'dist_normalize': False, 'clip_gm': False, 'clip_min': 0, 'clip_max': 5, 'tmap_stats': False, 'ensemble_step': 0.5, 'ground_metric_eff': False, 'reg': 0.01, 'acts_num_s

c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\ot\lp\__init__.py:354: UserWarning: numItermax reached before optimality. Try to increase numItermax.
  result_code_string = check_result(result_code)


At layer idx 8 and shape torch.Size([4096, 25088]), the OT cost is  3.888941482242103
Tmap stats (before correction) 
: For layer model.classifier.0.weight, frobenius norm from the joe's transport map is 0.015621191821992397
shape of T_var is  torch.Size([4096, 4096])
T_var before correction  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
T_var after correction  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
T_var stats: max 0.9995905756950378, min 0.0, mean 0.0002440406387904659, std 0.015616696327924728 
the transport map is  tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0.,

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


...

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


Files already downloaded and verified
Files already downloaded and verified


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\lightning\pytorch\trainer\connectors\data_connector.py:441: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=23` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 20/20 [00:01<00:00, 11.77it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      val_accuracy          0.7163000106811523
        val_loss             1.801663875579834
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


epoch,▁
trainer/global_step,▁
val_accuracy,▁
val_loss,▁
epoch,0
trainer/global_step,0
val_accuracy,0.7163
val_loss,1.80166


In [6]:
save_path = r'C:\Users\filos\OneDrive\Desktop\ETH\model-fusion\VGG_cifar10_models'
modelOTF_name = 'modelOTF_512AB_VGG.t7'
torch.save(ot_fused_model.state_dict(), save_path + '\\' + modelOTF_name)

In [9]:
# LMC barrier
print("------- Computing LMC barrier after alignment -------")

lmc_experiment.run_lmc(
    datamodule_type=datamodule_type,
    modelA=modelA_aligned,
    modelB=modelB,
    granularity=21
)

# Losses for ot fusion model and vanilla averaging model
datamodule_hparams_lmc = {'batch_size': 1024, 'data_dir': BASE_DATA_DIR}
datamodule_lmc = datamodule_type.get_data_module(**datamodule_hparams)
datamodule_lmc.prepare_data()
datamodule_lmc.setup('fit')

vanilla_loss = lmc_utils.compute_loss(vanilla_averaging_model, datamodule_lmc)
fused_loss = lmc_utils.compute_loss(ot_fused_model, datamodule_lmc)

print(f"Vanilla loss pre fine-tuning: {vanilla_loss}")
print(f"Fused loss pre fine-tuning: {fused_loss}")

Seed set to 42


------- Computing LMC barrier after alignment -------
Files already downloaded and verified
Files already downloaded and verified


Seed set to 42
Seed set to 42


Alpha: 0.00 (model 2), Train average loss: 0.17971 Train barrier:  0


Seed set to 42


...

Alpha: 0.85, Train average loss: 0.48993 Train barrier 0.38774431275718746


Seed set to 42


Alpha: 0.90, Train average loss: 0.26514 Train barrier 0.167516056314177
Alpha: 0.95, Train average loss: 0.13553 Train barrier 0.042462729076743116
Loss model 1: 0.08851, Loss model 2: 0.17971, Alpha argmax: 0.50000
Barrier: 1.64329
Files already downloaded and verified
Files already downloaded and verified


Seed set to 42
Seed set to 42


Vanilla loss pre fine-tuning: 2.302413641145494
Fused loss pre fine-tuning: 1.7775452330271404


In [12]:
# Pyhessian (compute sharpness and eigenspectrum of base models, vanilla avg, ot fusion and finetuned solutions)
print("------- Computing sharpness -------")

print("------- Model A -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=modelA, compute_density=False, figure_name='modelA.pdf') 

print("------- Model B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=modelB, compute_density=False, figure_name='modelB.pdf')

print("------- Model A aligned to B -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=modelA_aligned,  compute_density=False, figure_name='modelA_aligned.pdf')

print("------- Vanilla avg model -------")
vanilla_averaging_model.to('cuda')
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=vanilla_averaging_model,  compute_density=False, figure_name='vanilla_avg.pdf')

print("------- OT fusion model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=ot_fused_model,  compute_density=False, figure_name='otmodel_same_init32_VGG.pdf')

Seed set to 42


------- Computing sharpness -------
------- Model A -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 6.3732


Seed set to 42



***Trace:  367.128670828683
------- Model B -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 8.2340


Seed set to 42



***Trace:  540.820805140904
------- Model A aligned to B -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 6.3740


Seed set to 42



***Trace:  370.30755337801844
------- Vanilla avg model -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is -0.0057


KeyboardInterrupt: 

In [5]:
print("------- Vanilla avg model -------")
vanilla_averaging_model.to('cuda')
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type,model=vanilla_averaging_model,  compute_density=False, figure_name='vanilla_avg.pdf')

print("------- OT fusion model -------")
hessian_comp = pyhessian_experiment.run_pyhessian(datamodule_type=datamodule_type, model=ot_fused_model,  compute_density=False, figure_name='otmodel_same_init32_VGG.pdf')

------- Vanilla avg model -------


Seed set to 42


Files already downloaded and verified
Files already downloaded and verified


c:\Users\filos\OneDrive\Desktop\ETH\model-fusion\.venv\Lib\site-packages\torch\autograd\__init__.py:251: UserWarning: Using backward() with create_graph=True will create a reference cycle between the parameter and its gradient which can cause a memory leak. We recommend using autograd.grad when creating the graph to avoid this. If you have to use this function, make sure to reset the .grad fields of your parameters to None after use to break the cycle and avoid the leak. (Triggered internally at ..\torch\csrc\autograd\engine.cpp:1176.)
  Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


The top Hessian eigenvalue of this model is -0.0057


Seed set to 42



***Trace:  0.017302026637401935
------- OT fusion model -------
Files already downloaded and verified
Files already downloaded and verified
The top Hessian eigenvalue of this model is 3.9076

***Trace:  40.591278433799744


In [ ]:
save_path = r'C:\Users\filos\OneDrive\Desktop\ETH\model-fusion\Vgg_cifar10_models'

modelB_name = 'model512B_VGG.t7'
torch.save(modelB.state_dict(), save_path + '\\' + modelB_name)

modelA_aligned_name = 'model512A_aligned_to512B_VGG.t7'
torch.save(modelA_aligned.state_dict(), save_path + '\\' + modelA_aligned_name)